# M4 Featurizing with POS for each association and counting the individual appearences of each POS

In [ ]:
#import os
#os.environ["CUDA_VISIBLE_DEVICES"]="1" # gpu usage
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

## POSTable with Amounts of associated lemma included (tokenbased) with NA tag!!!

In [ ]:
# Treetagging in shell and saving output in txt file

# DONE IN TreeTagger directory for right working directory


In [ ]:
# STANZA problem: not good for single word tagging, tags in sentence relations and therefore unknown words are not labelled as X but NN mostly

#import tensorflow
#import stanza
words = ['tokens', 'apple', 'vfkkelwk', 'what', 'laughing', 'bowl']
nlp = stanza.Pipeline(model_dir='/users/aylin.wahl/stanza_resources',processors='tokenize, pos',tokenize_pretokenized= True,download_method=None)
doc = nlp([words])
print(*[f'word: {word.text}\tupos: {word.upos}\txpos: {word.xpos}\tfeats: {word.feats if word.feats else "_"}' for sent in doc.sentences for word in sent.words], sep='\n')

In [ ]:
# SPACY problem: not running and does not tag words as unknown as well (spacy doe not import due to no menory (restarting hub helps)/ nect problem does not find library)

#!pip install spacy
#!python -m spacy download en_core_web_sm

# downgrading spacy
!pip install -U pip setuptools wheel
!pip install -U spacy==2.3.5
!python -m spacy download en_core_web_sm
# note: This error originates from a subprocess, and is likely not a problem with pip.ERROR: Failed building wheel for spacy Failed to build spacy ERROR: Could not build wheels for spacy, which is required to install pyproject.toml-based projects /usr/bin/python: No module named spacy

#!pip install -U spacy==2.3.5
#!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz

#!pip install -U pip setuptools wheel
#!pip install -U spacy
#!python -m spacy download en_core_web_sm
#!pip install resume-parser
#!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz
!pip install importlib-metadata==3.2.0

import spacy
from spellchecker import SpellChecker

nlp = spacy.load('en', disable=['parser', 'ner'])

spell = SpellChecker()

if __name__ == '__main__':
    doc = nlp("He who values value large column sbxdata actual maximum ptsavatar")

    known = set(spell.known([token.text.lower() for token in doc]))

    for token in doc:
        if token.text.lower() in known:
            pos = token.pos_
        else:
            pos = ""

        print(f'{pos:5s} {token}')     
"""
import spacy
nlp = spacy.load('en_core_web_sm')

words = ['tokens', 'apple', 'vfkkelwk', 'what', 'laughing', 'bowl']

# disable everything except the tagger
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "tagger"]
nlp.disable_pipes(*other_pipes)

# use nlp.pipe() instead of nlp() to process multiple texts more efficiently
for doc in nlp.pipe(words):
    if len(doc) > 0:
        print(doc[0].text, doc[0].tag_)
"""

## POSTable with Amounts of associated lemma included (tokenbased)

In [ ]:
 # adding row to asscotable with POS information 

import nltk
from nltk.corpus import brown
from nltk.tag import UnigramTagger
nltk.download('universal_tagset')
import numpy as np
import pandas as pd

#getting the possible tags for brown corpus
#nltk.download('tagsets')
#nltk.help.brown_tagset()

#datatype= 'extreme'
datatype= 'filtered'
#wordclass = 'adj'
wordclass = 'nouns'
#wordclass = 'verbs'

assoctable = pd.read_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_'+datatype+'/noNAtypes_'+wordclass+'_'+datatype+'_AssocTable.csv')

'''
# POS tagging un known words as Nouns
#insert POS row in index position 0 in the assoctable
POSrow = []
tokens = list(assoctable.columns)
pos_tags = nltk.pos_tag(tokens) #PennTreeBank machinelearnaing tagger of-the-shelf
for tup,POS in pos_tags:  #pos from tuples into list
    POSrow.append(POS)
assoctable.loc[-1]=POSrow  #POSlist into dataframe
assoctable = assoctable.sort_index().reset_index(drop=True)  #sorting dataframe with POS row
assoctable.iloc[0,0]=''  #targets no rowwise POS
assoctable.iloc[0,1]=''  #CS no POS
'''

# POS tagging unknown words as 'None'
#insert POS row in index position 0 in the assoctable
POSrow = []
tokens = list(assoctable.columns)
tagger = UnigramTagger(brown.tagged_sents(tagset = 'universal'))   #training data based unigram tagging, with brown corpus trained tagset='universal'
for word, tag in tagger.tag(tokens):
    POSrow.append(tag)
assoctable.loc[-1]=POSrow  #POSlist into dataframe
assoctable = assoctable.sort_index().reset_index(drop=True)  #sorting dataframe with POS row
assoctable.loc[0] =  assoctable.loc[0].replace([None], 'None')# making None POS to a string
assoctable.iloc[0,0]=''  #targets no rowwise POS
assoctable.iloc[0,1]=''  #CS no POS
# too many unknown tags?

assoctable.to_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/M4_POStable/NoUnknownNoNAtypes_'+wordclass+'_'+datatype+'_POStableAssoccount.csv',index=False)   
# counting the amount of None tagged associations (without token count) and listing them to get an insight which words are missing

print('All Associations: '+str(len(list(assoctable.loc[0]))-2)) # all associations
print('None Associations: '+str(list(assoctable.loc[0]).count('None'))) #count of None associations
none = list(assoctable.apply(lambda row: row[row == 'None'].index, axis=1))  # list of associations that are None tagged
nonelist = []
for x in range(len(none[0])):
    nonelist.append(none[0][x])
with open('/compLing/students/hiwi-theses/projects/aylin.wahl/M4/'+wordclass+'_NoneAssociations.txt', "a") as f:
    print('All Associations: '+str(len(list(assoctable.loc[0]))-2),file=f) # all associations
    print('None Associations: '+str(list(assoctable.loc[0]).count('None')),file=f)
    for assoc in nonelist:
        print(assoc, file=f)


In [5]:
POStable

,CS,.,DET,ADV,ADP,PRON,X,None,NOUN,PRT,ADJ,NUM,NaN,VERB,NaN,CONJ
1,4.31,0,0,1,0,0,0,21,216,0,22,0,0,31,0,1
2,2.34,0,13,18,8,0,0,21,171,20,16,3,0,16,0,0
3,2.59,0,13,10,4,0,3,51,76,5,51,49,0,20,0,1
4,4.93,0,0,5,2,0,0,29,178,0,21,0,0,58,0,0
5,4.64,0,3,0,2,0,0,31,209,8,30,0,0,6,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3694,2.11,0,0,4,0,0,0,44,152,0,68,2,0,5,0,0
3695,4.59,0,0,0,1,0,0,46,171,0,16,0,0,13,0,0
3696,4.20,0,0,1,3,0,0,47,178,0,9,1,0,5,0,0
3697,4.67,0,1,1,0,0,0,38,223,0,14,0,0,13,0,0


In [1]:
# counting POStags for each target (takes forever!!!!!!) only needed once
import pandas as pd
import numpy as np
from collections import Counter

#wordclass = 'adj'
#wordclass = 'nouns'
wordclass = 'verbs'

assoctable = pd.read_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/M4_POStable/NoUnknownNoNAtypes_'+wordclass+'_filtered_POStableAssoccount.csv', low_memory=False)

#countsALL = Counter( tag for word,  tag in pos_tags) #counting tags for whole asssoctable
#POStagsALL = list(countsALL)         #list of tags for all assoc in assoctable
# gettig all POS tags used in this process from the  brown tagger (browntagset)
POStagsALL = list(set(list(assoctable.loc[0])))
POStable = pd.DataFrame(data=assoctable['CS'], index=list(assoctable.index), columns=['CS']+POStagsALL)
POStable = POStable.replace([None], 0)
POStable = POStable.drop(0,axis=0)

assoctable.iloc[1:,1:] = assoctable.iloc[1:,1:].astype(int)
associationlist = list(assoctable.iloc[1:,2:].columns)
# counting the POS amount per target with tokenamount
for index, row in assoctable.iloc[1:,2:].iterrows():
    indexesnonzero = np.nonzero(list(row))  #getting the indexes of the nonzero values to count them individually
    for ind in range(0,len(list(indexesnonzero)[0])):
        i = list(indexesnonzero)[0][ind] #index of the column with nonzero value
        column = associationlist[i]      #getting the association with a nonzero value
        pos = assoctable.loc[0,column]   # getting POS of the word
        amount = assoctable.iloc[1:,2:].loc[index,column]# getting the amount to add to the postable
        POStable.loc[index,pos] += amount
          
POStable.to_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/M4_POStable/NoUnknownNoNAtypes_'+wordclass+'_filtered_POStable.csv',index=False)   

print(POStable.sum(axis=1))

1      282.25
2      288.56
3      287.72
4      277.38
5      283.68
        ...  
765    226.68
766    222.33
767    256.17
768    253.64
769    267.55
Length: 769, dtype: float64


## POStable without Amounts of associated lemmas (typebased)

In [ ]:
POStable.to_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/M4_POStable/NoUnknownNoNAtypes_'+wordclass+'_filtered_POStable.csv',index=False)   

print(POStable.sum(axis=1))

In [ ]:
POStable

In [ ]:
# POStagging all associations and counting the N/V/A/Other in the associated lemmas for a target

 # adding row to asscotable with POS information 
import nltk
from nltk.corpus import brown
from nltk.tag import UnigramTagger
import numpy as np
import pandas as pd

#datatype= 'extreme'
datatype= 'filtered'
#wordclass = 'adj'
wordclass = 'nouns'
#wordclass = 'verbs'

assoctable = pd.read_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_'+datatype+'/noNAtypes_'+wordclass+'_'+datatype+'_AssocTable.csv')

# POS tagging un known words as Nouns
#insert POS row in index position 0 in the assoctable
POSrow = []
tokens = list(assoctable.columns)
pos_tags = nltk.pos_tag(tokens) #PennTreeBank machinelearnaing tagger of-the-shelf
for tup,POS in pos_tags:  #pos from tuples into list
    POSrow.append(POS)
assoctable.loc[-1]=POSrow  #POSlist into dataframe
assoctable = assoctable.sort_index().reset_index(drop=True)  #sorting dataframe with POS row
assoctable.iloc[0,0]=''  #targets no rowwise POS
assoctable.iloc[0,1]=''  #CS no POS

#setting up POStable
from collections import Counter

countsALL = Counter( tag for word,  tag in pos_tags) #counting tags for whole asssoctable
POStagsALL = list(countsALL)         #list of tags for all assoc in assoctable
POStable = pd.DataFrame(index=list(assoctable.index), columns=['Unnamed: 0','CS']+POStagsALL)
POStable['CS'] = assoctable['CS']
POStable['Unnamed: 0'] = assoctable['Unnamed: 0']
POStable = POStable.replace([None], 0)
POStable = POStable.drop([0],axis=0)

#filling POStable dataframe with counts of the POS
for ind in range(1,len(assoctable.index)):        
    #target = list(assoctable.iloc[ind])
    nonzeros = assoctable.iloc[ind].to_numpy().nonzero()  #getting a list of the indexes of the nonzero values
    nonzeros = nonzeros[0].tolist()
    nonzeros.remove(0)# deleting 0 and 1 value for targetname and CS
    nonzeros.remove(1)
    for i in nonzeros:
        #non = nonzeros[i]
        POS = assoctable[assoctable.columns[i]][0] #getting POS of nonzero association
        POStable.loc[ind,POS] += 1 # counting +1 for this POS in the target row of POStable

POStable.to_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/M4_POStable/noNAtypes_'+wordclass+'_'+datatype+'_POStableLemmacount.csv',index=False)   

print(POStable.sum(axis=1))
    
    

In [ ]:
POStable

# Normalizing POStable

In [ ]:
# L1 normalization by dividing amount of associated POS by amount of all associated POS
import pandas as pd

wordclass = 'nouns'
#wordclass = 'adj'
#wordclass = 'verbs'

df = pd.read_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/M4_POStable/noNAtypes_'+wordclass+'_filtered_POStable.csv')
df = df.drop(df.index[0]) #delete whole amount of POS association row
df.reset_index(inplace=True) #reset index
cs = df['CS']
df = df.drop(['CS'],axis=1) #drop CS to exclude from normalization

df = df.div(df.sum(axis=1),axis=0)
df.insert(0, 'CS', cs) #adding back Concreteness scores
t = pd.read_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_filtered/noNAtypes_'+wordclass+'_filtered_AssocTable.csv')
df.insert(0,'Unnamed: 0', t['Unnamed: 0']) #adding targets whihc got lost through POS counting

df.to_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/M4_POStable/NORMnoNAtypes_'+wordclass+'_filtered_POStable.csv',index=False)
